In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00


In [2]:
import os
os.listdir()  # Check if the model file is present


['.config', 'deeplabv3_breast_ultrasound.pth', 'sample_data']

In [5]:
import torch

model_path = "/content/deeplabv3_breast_ultrasound.pth"

try:
    checkpoint = torch.load(model_path, map_location="cpu")
    print("✅ Model file is valid!")
except Exception as e:
    print("❌ Model file is corrupted or invalid:", e)


❌ Model file is corrupted or invalid: PytorchStreamReader failed reading zip archive: failed finding central directory


In [8]:
import torch

model_path = "/content/deeplabv3_breast_ultrasound (2).pth"

# Load DeepLabV3+ model
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=False)
model.classifier[4] = torch.nn.Conv2d(256, 2, kernel_size=(1,1))  # Adjust output classes

# Load state_dict and remove aux_classifier keys
checkpoint = torch.load(model_path, map_location="cpu")

filtered_state_dict = {k: v for k, v in checkpoint.items() if "aux_classifier" not in k}

# Load the modified state_dict
model.load_state_dict(filtered_state_dict, strict=False)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("✅ Model loaded successfully without aux_classifier!")


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


✅ Model loaded successfully without aux_classifier!


In [9]:
def segment_tumor(image):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)['out']

    pred_mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()

    return pred_mask


In [ ]:
import numpy as np

def process_image(image):
    pred_mask = segment_tumor(image)
    return np.uint8(pred_mask * 255)  # Convert to grayscale

gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Image(type="numpy"),
    title="Breast Tumor Segmentation",
    description="Upload a breast ultrasound image, and the model will segment the tumor region.",
    allow_flagging="never"
).launch(debug=True)


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://789f99c46c04153cab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1663, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^